In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
DIR = './processed/'
DATA = './data/'

In [ ]:
# redefine the calculation of bias and centred RSMD since skillmetrics does not deal with NaNs
# (thanks ChatGPT for a clean code!)
def bias_urmsd(x: pd.Series, y: pd.Series) -> float:
    """
    Calculate the mean bias and the centered (unbiased) 
    Root Mean Square Difference between two Pandas Series.
    NaNs are excluded pairwise.

    Parameters:
        x (pd.Series): First dataset (predictions)
        y (pd.Series): Second dataset (reference)

    Returns:
        float: The centered RMSD value
    """
    import pandas as pd
    import numpy as np

    # Align the series and drop NaNs pairwise
    aligned = pd.concat([x, y], axis=1).dropna()
    x_clean, y_clean = aligned.iloc[:, 0], aligned.iloc[:, 1]
    
    if len(x_clean) == 0:
        raise ValueError("No valid data points after removing NaNs.")

    # Bias
    bias = x_clean - y_clean
    
    # Remove the means
    x_anom = x_clean - x_clean.mean()
    y_anom = y_clean - y_clean.mean()
    
    # Compute centered RMSD
    diff = x_anom - y_anom
    rmsd_centered = np.sqrt(np.mean(diff**2))
    
    return bias.mean(),rmsd_centered

In [ ]:
# Load CMIP SI edges and define best models (from other script)
dfj=pd.read_csv(DIR+'CMIP6-26_SIedge_Jan_1930_1939.csv',index_col=0)
dff=pd.read_csv(DIR+'CMIP6-26_SIedge_Feb_1930_1939.csv',index_col=0)
dfn=pd.read_csv(DIR+'CMIP6-26_SIedge_Nov_1930_1939.csv',index_col=0)
dfd=pd.read_csv(DIR+'CMIP6-26_SIedge_Dec_1930_1939.csv',index_col=0)

best8 = ['FIO-ESM-2-0',
 'IPSL-CM6A-LR-INCA',
 'TaiESM1',
 'IPSL-CM6A-LR',
 'CMCC-CM2-HR4',
 'ACCESS-ESM1-5',
 'GFDL-ESM4',
 'CanESM5']

In [ ]:
# Load CMIP SI piControl edges
dfj_pi=pd.read_csv(DIR+'CMIP6-21_SIedge_Jan_piControl.csv',index_col=0)
dff_pi=pd.read_csv(DIR+'CMIP6-21_SIedge_Feb_piControl.csv',index_col=0)
dfn_pi=pd.read_csv(DIR+'CMIP6-21_SIedge_Nov_piControl.csv',index_col=0)
dfd_pi=pd.read_csv(DIR+'CMIP6-21_SIedge_Dec_piControl.csv',index_col=0)

In [ ]:
# Mackintosh (1972) sea-ice edge
mkN = pd.read_csv(DATA+'MK72N.csv',names=['lon','lat'])
mkD = pd.read_csv(DATA+'MK72D.csv',names=['lon','lat'])
mkJ = pd.read_csv(DATA+'MK72J.csv',names=['lon','lat'])
mkF = pd.read_csv(DATA+'MK72F.csv',names=['lon','lat'])

In [ ]:
# Load whale catches and group them according to longitudinal bands
df = pd.read_csv(DIR+'hump_filled_RR.csv', sep= ';')
df = df[df['lat'] <= -40]
#%% Decades Chosen
Decade1930 = df[(df['year'] >= 1930) & (df['year'] < 1940)]
## Create 10 deg longitudinal group labels
labels = ['{0} - {1}'.format(i, i + 9) for i in range(-179, 180, 10)]

def get_group(df, name, obj=None, default=None) :
    # extract a group from a dataframe given a categorical name
    if obj is None :
        obj = df.obj
    try :
        inds = df.indices[name]
    except KeyError:
        if default is None :
            raise
        inds = default
    return df.obj.take(inds, df.axis)

def month_groups(decade,month,labels):
    # create longitudinal groups for a given month
    sel = decade[decade['month']==month]
    sel = sel[sel['class'] <7]
    sel['group'] = pd.cut(sel['long'], range(-180, 185, 10), right=False, labels=labels)
    groups = sel.groupby('group', observed=False)
    return groups

dict_groups = ['{0}\n{1}'.format(i, i + 9) for i in range(-180, 180, 10)]

In [ ]:
#%% NOVEMBER
groupm = month_groups(Decade1930,11,labels)
# create lists of arrays of latitudinal positions and catches, each one for every 10 degrees bin
LN = [] # latitudes
NN = [] # number of whale catches
for l in labels:
    group = get_group(groupm, l, default = None)
    LN.append(group['lat'].values)
    NN.append(str(sum(group['hbk'])))

#%% DECEMBER
groupm = month_groups(Decade1930,12,labels)
# create lists of arrays of latitudinal positions and catches, each one for every 10 degrees bin
LD = [] # latitudes
ND = [] # number of whale catches
for l in labels:
    group = get_group(groupm, l, default = None)
    LD.append(group['lat'].values)
    ND.append(str(sum(group['hbk'])))

#%% JANUARY
groupm = month_groups(Decade1930,1,labels)
# create lists of arrays of latitudinal positions and catches, each one for every 10 degrees bin
LJ = [] # latitudes
NJ = [] # number of whale catches
for l in labels:
    group = get_group(groupm, l, default = None)
    LJ.append(group['lat'].values)
    NJ.append(str(sum(group['hbk'])))

#%% FEBRUARY
groupm = month_groups(Decade1930,2,labels)
# create lists of arrays of latitudinal positions and catches, each one for every 10 degrees bin
LF = [] # latitudes
NF = [] # number of whale catches
for l in labels:
    group = get_group(groupm, l, default = None)
    LF.append(group['lat'].values)
    NF.append(str(sum(group['hbk'])))

In [ ]:
# Generate Figure 4
fig, axs = plt.subplots(4, 2, figsize=(15,12), sharey=True, sharex = True, tight_layout=True )
plt.subplots_adjust
x = np.arange(-180,180,10)
OS = 2
FS = 8
LAT = -70

##################### all models column
ax = axs[0,0]
M = dfn.median(axis=1,skipna=True)
p75 = dfn.quantile(0.75,axis=1)
p25 = dfn.quantile(0.25,axis=1)
M.plot(ax=ax,label='November Median (all)')
p75.plot(ax=ax,color='C0',linestyle='--',label='75th perc.')
p25.plot(ax=ax,color='C0',linestyle='-.',label='25th perc.')
# pre-industrial
M = dfn_pi.median(axis=1,skipna=True)
p75 = dfn_pi.quantile(0.75,axis=1)
p25 = dfn_pi.quantile(0.25,axis=1)
M.plot(ax=ax,color='black', linestyle='-', alpha=0.4,label='pre-industrial')
ax.fill_between(M.index, p75, p25, facecolor='grey', alpha=0.1)
out = ax.boxplot(LN,widths = 7, positions = range(-180,180,10), zorder=10)
#ax.legend(loc='upper right')
for i in range(11,34):
    ax.text(x[i]-OS,LAT,NN[i],color='grey',fontsize=FS)

ax = axs[1,0]
M = dfd.median(axis=1,skipna=True)
p75 = dfd.quantile(0.75,axis=1)
p25 = dfd.quantile(0.25,axis=1)
M.plot(ax=ax,label='December Median (all)')
p75.plot(ax=ax,color='C0',linestyle='--',label='75th perc.')
p25.plot(ax=ax,color='C0',linestyle='-.',label='25th perc.')
# pre-industrial
M = dfd_pi.median(axis=1,skipna=True)
p75 = dfd_pi.quantile(0.75,axis=1)
p25 = dfd_pi.quantile(0.25,axis=1)
M.plot(ax=ax,color='black', linestyle='-', alpha=0.4,label='pre-industrial')
ax.fill_between(M.index, p75, p25, facecolor='grey', alpha=0.1)
out = ax.boxplot(LD,widths = 7, positions = range(-180,180,10), zorder=10)
#ax.legend(loc='upper right')
for i in range(11,34):
    ax.text(x[i]-OS,LAT,ND[i],color='grey',fontsize=FS)

ax = axs[2,0]
M = dfj.median(axis=1,skipna=True)
p75 = dfj.quantile(0.75,axis=1)
p25 = dfj.quantile(0.25,axis=1)
M.plot(ax=ax,label='January Median (all)')
p75.plot(ax=ax,color='C0',linestyle='--',label='75th perc.')
p25.plot(ax=ax,color='C0',linestyle='-.',label='25th perc.')
# pre-industrial
M = dfj_pi.median(axis=1,skipna=True)
p75 = dfj_pi.quantile(0.75,axis=1)
p25 = dfj_pi.quantile(0.25,axis=1)
M.plot(ax=ax,color='black', linestyle='-', alpha=0.4,label='pre-industrial')
ax.fill_between(M.index, p75, p25, facecolor='grey', alpha=0.1)
out = ax.boxplot(LJ,widths = 7, positions = range(-180,180,10), zorder=10)
#ax.legend(loc='upper right')
for i in range(11,34):
    ax.text(x[i]-OS,LAT,NJ[i],color='grey',fontsize=FS)

ax = axs[3,0]
M = dff.median(axis=1,skipna=True)
p75 = dff.quantile(0.75,axis=1)
p25 = dff.quantile(0.25,axis=1)
M.plot(ax=ax,label='Median (all)')
p75.plot(ax=ax,color='C0',linestyle='--',label='75th perc.')
p25.plot(ax=ax,color='C0',linestyle='-.',label='25th perc.')
# pre-industrial
M = dff_pi.median(axis=1,skipna=True)
p75 = dff_pi.quantile(0.75,axis=1)
p25 = dff_pi.quantile(0.25,axis=1)
M.plot(ax=ax,color='black', linestyle='-', alpha=0.4,label='pre-industrial')
ax.fill_between(M.index, p75, p25, facecolor='grey', alpha=0.1)
out = ax.boxplot(LF,widths = 7, positions = range(-180,180,10), zorder=10)
ax.legend(loc='upper right')
for i in range(11,34):
    ax.text(x[i]-OS,LAT,NF[i],color='grey',fontsize=FS)

##################### best models column
ax = axs[0,1]
M = dfn[best8].median(axis=1,skipna=True)
p75 = dfn[best8].quantile(0.75,axis=1)
p25 = dfn[best8].quantile(0.25,axis=1)
M.plot(ax=ax,color='C1',label='November Median (high-rank)')
p75.plot(ax=ax,color='C1',linestyle='--',label='75th perc.')
p25.plot(ax=ax,color='C1',linestyle='-.',label='25th perc.')
ax.plot(mkN.lon,mkN.lat,color='C3',label='Mackintosh (1972)')
out = ax.boxplot(LN,widths = 7, positions = range(-180,180,10), zorder=10)
#ax.legend(loc='upper right')
for i in range(11,34):
    ax.text(x[i]-OS,LAT,NN[i],color='grey',fontsize=FS)

ax = axs[1,1]
M = dfd[best8].median(axis=1,skipna=True)
p75 = dfd[best8].quantile(0.75,axis=1)
p25 = dfd[best8].quantile(0.25,axis=1)
M.plot(ax=ax,color='C1',label='December Median (high-rank)')
p75.plot(ax=ax,color='C1',linestyle='--',label='75th perc.')
p25.plot(ax=ax,color='C1',linestyle='-.',label='25th perc.')
ax.plot(mkD.lon,mkD.lat,color='C3',label='Mackintosh (1972)')
out = ax.boxplot(LD,widths = 7, positions = range(-180,180,10), zorder=10)
#ax.legend(loc='upper right')
for i in range(11,34):
    ax.text(x[i]-OS,LAT,ND[i],color='grey',fontsize=FS)

ax = axs[2,1]
M = dfj[best8].median(axis=1,skipna=True)
p75 = dfj[best8].quantile(0.75,axis=1)
p25 = dfj[best8].quantile(0.25,axis=1)
M.plot(ax=ax,color='C1',label='January Median (high-rank)')
p75.plot(ax=ax,color='C1',linestyle='--',label='75th perc.')
p25.plot(ax=ax,color='C1',linestyle='-.',label='25th perc.')
ax.plot(mkJ.lon,mkJ.lat,color='C3',label='Mackintosh (1972)')
out = ax.boxplot(LJ,widths = 7, positions = range(-180,180,10), zorder=10)
#ax.legend(loc='upper right')
for i in range(11,34):
    ax.text(x[i]-OS,LAT,NJ[i],color='grey',fontsize=FS)

ax = axs[3,1]
M = dff[best8].median(axis=1,skipna=True)
p75 = dff[best8].quantile(0.75,axis=1)
p25 = dff[best8].quantile(0.25,axis=1)
M.plot(ax=ax,color='C1',label='Median (high-rank)')
p75.plot(ax=ax,color='C1',linestyle='--',label='75th perc.')
p25.plot(ax=ax,color='C1',linestyle='-.',label='25th perc.')
ax.plot(mkF.lon,mkF.lat,color='C3',label='Mackintosh (1972)')
out = ax.boxplot(LF,widths = 7, positions = range(-180,180,10), zorder=10)
ax.legend(loc='upper right')
for i in range(11,34):
    ax.text(x[i]-OS,LAT,NF[i],color='grey',fontsize=FS)

# add annotations
for ax in axs.flat:
    ax.set(xlabel='Longitude bands (degrees)', ylabel='Latitude (degrees)')
    ax.xaxis.get_label().set_fontsize(13)
    ax.yaxis.get_label().set_fontsize(13)
    ax.set_xticks(x)
    ax.set_xticklabels(dict_groups)
    ax.xaxis.set_tick_params(labelsize=10)
    ax.yaxis.set_tick_params(labelsize=11)
    ax.set_xlim([-70, 150]) ## Atlantic and Indian Ocean
    ax.set_ylim([-71,-47])
    ax.label_outer()

axs[0,0].set_title('a) November (1930-39)', loc = 'left', fontsize = 12)
axs[1,0].set_title('b) December (1930-39)', loc = 'left', fontsize = 12)
axs[2,0].set_title('c) January (1930-39)', loc = 'left', fontsize = 12)
axs[3,0].set_title('d) February (1930-39)', loc = 'left', fontsize = 12)
plt.savefig('figures/Fig4.png',dpi=300)

In [ ]:
# more quantitative way to visualize the relationship using 
# 10 degrees bins' median lats from catches and models
labels = ['{0} - {1}'.format(i, i + 9) for i in range(-180, 180, 10)]

# the following functions extract the bins from the whale data and CMIP.
# prevent SettingWithCopyWarning message from appearing
pd.options.mode.chained_assignment = None

def Wlon10_median(decade,month):
    # create longitudinal groups for a given month
    sel = decade[decade['month']==month]
    sel = sel[sel['class'] < 7]
    sel['lon_range10'] = pd.cut(sel['long'], range(-180, 185, 10), right=True, labels=labels)
    bins = sel['lat'].groupby(sel['lon_range10'], observed=False).mean()
    std = sel['lat'].groupby(sel['lon_range10'], observed=False).std()
    return bins,std

def CMIPlon10_median(dfC):
    dfC['M'] = dfC.median(axis=1,skipna=True)
    dfC['lon_range10'] = pd.cut(dfC.index, range(-180, 185, 10), right=True, labels=labels)
    CMIP = dfC['M'].groupby(dfC['lon_range10'], observed=False).mean()
    std = dfC['M'].groupby(dfC['lon_range10'], observed=False).std()
    return CMIP,std

# CMIP median
dfC = pd.read_csv(DIR+'CMIP6-26_SIedge_Nov_1930_1939.csv',index_col=0)
dfc = dfC[best8]
CMIPn,CMIPns = CMIPlon10_median(dfc)
dfC = pd.read_csv(DIR+'CMIP6-26_SIedge_Dec_1930_1939.csv',index_col=0)
CMIPd,CMIPds = CMIPlon10_median(dfC[best8])
dfC = pd.read_csv(DIR+'CMIP6-26_SIedge_Jan_1930_1939.csv',index_col=0)
CMIPj,CMIPjs = CMIPlon10_median(dfC[best8])
dfC = pd.read_csv(DIR+'CMIP6-26_SIedge_Feb_1930_1939.csv',index_col=0)
CMIPf,CMIPfs = CMIPlon10_median(dfC[best8])

# whaling records
Decade1930 = df[(df['year'] >= 1930) & (df['year'] < 1940)]
Wn,Wns = Wlon10_median(Decade1930,11)
Wd,Wds = Wlon10_median(Decade1930,12)
Wj,Wjs = Wlon10_median(Decade1930,1)
Wf,Wfs = Wlon10_median(Decade1930,2)

# group the variables for plotting and calculating mean bias and centred RMSD
Y = [Wn,Wd,Wj,Wf]
X = [CMIPn,CMIPd,CMIPj,CMIPf]
B = []
CR = []
for pred,ref in zip(X,Y):
    bias,urmsd = bias_urmsd(pred,ref)
    B.append(np.round(bias,1))
    CR.append(np.round(urmsd,1))


In [ ]:
# Create the dataframe and save the CSV
df = pd.concat([pd.Series(NN,index=Wn.index),Wn,Wns,
                pd.Series(ND,index=Wd.index),Wd,Wds,
                pd.Series(NJ,index=Wj.index),Wj,Wjs,
                pd.Series(NF,index=Wf.index),Wf,Wfs,
                CMIPn,CMIPns,CMIPd,CMIPds,CMIPj,CMIPjs,CMIPf,CMIPfs],axis=1)
df.columns = ['catchnum_N','catchlat_N','catchlat_N_std','catchnum_D','catchlat_D','catchlat_D_std',
              'catchnum_J','catchlat_J','catchlat_J_std','catchnum_F','catchlat_F','catchlat_F_std',
              'CMIP6lat_N','CMIP6lat_N_std','CMIP6lat_D','CMIP6lat_D_std',
              'CMIP6lat_J','CMIP6lat_J_std','CMIP6lat_F','CMIP6lat_F_std',
             ]
df.to_csv(DIR+'whales-CMIP6-seaice-edge-1930-39.csv')

In [ ]:
# Generate Figure 5
dlat = [-70,-50]
f,ax = plt.subplots(1,1)
ax.plot(dlat,dlat,'k',linewidth=0.8)
ax.plot(dlat,np.array(dlat)+2,'grey',linewidth=0.5)
ax.plot(dlat,np.array(dlat)+4,'grey',linewidth=0.5)
ax.plot(dlat,np.array(dlat)-2,'grey',linewidth=0.5)
ax.plot(dlat,np.array(dlat)-4,'grey',linewidth=0.5)

ax.errorbar(X[0], Y[0], yerr = CMIPns, xerr = Wns, 
             fmt ='o',label=f'November ({B[0]:.1f},{CR[0]:.1f})',
           elinewidth = 0.5, markeredgecolor = 'black')
ax.errorbar(X[1], Y[1], yerr = CMIPds, xerr = Wds, 
             fmt ='o',label=f'December ({B[1]:.1f},{CR[1]:.1f})',
           elinewidth = 0.5, markeredgecolor = 'black')
ax.errorbar(X[2], Y[2], yerr = CMIPjs, xerr = Wjs, 
             fmt ='o',label=f'January     ({B[2]:.1f},{CR[2]:.1f})',
           elinewidth = 0.5, markeredgecolor = 'black')
ax.errorbar(X[3], Y[3], yerr = CMIPfs, xerr = Wfs, 
             fmt ='o',label=f'February   ({B[3]:.1f},{CR[3]:.1f})',
           elinewidth = 0.5, markeredgecolor = 'black')
ax.set_xlim(dlat)
ax.set_ylim(dlat)
ax.set_aspect('equal')
plt.title('1930-1939')
plt.xlabel('CMIP6 sea-ice edge (degrees)')
plt.ylabel('Historical whale catches (degrees)')
plt.legend(loc='upper left')
plt.savefig('figures/Fig5.png',dpi=300)
plt.show()